In [14]:
import os, shutil, fnmatch, datetime, openpyxl
from openpyxl import Workbook, load_workbook

#los ficheros de audio los ponemos en dir_raiz_audios
#establecemos las personas que van a transcribir

#lista_personas = ('itzileguinazabal','jaimerequejo', 'santiagosegarra')
lista_personas = ('santiagosegarra',)

#establecemos el numero de ficheros por persona

num_audios_por_persona = 10

#creamos las estructuras de directorios

dir_raiz_audios = '/Users/jaimerequejo/Box Sync/STT AEAT/Audios/'
dir_log_audios = '/Users/jaimerequejo/Box Sync/STT AEAT/Audios/Transcripciones/logs/'
dir_sources_audios = '/Users/jaimerequejo/Box Sync/STT AEAT/Audios/Sources/'
dir_sources_personas = '/Users/jaimerequejo/Box Sync/STT AEAT/Audios/Transcripciones/Personas/'
os.makedirs(dir_raiz_audios, exist_ok=True)
os.makedirs(dir_sources_audios, exist_ok=True)
os.makedirs(dir_sources_personas, exist_ok=True)
os.makedirs(dir_log_audios, exist_ok=True)
timeID = str(datetime.datetime.now()).replace('-', '').replace(' ', '_').replace(':', '').replace('.', '')
for persona in lista_personas:
    dir_persona = dir_sources_personas + '/' + persona + '/' + timeID
    os.makedirs(dir_persona, exist_ok=True)
    
#el el caso de querer distribuirlos todos, hacemos un ratio entre las personas en la lista de trasncriptores y el numero total de ficheros
#num_personas = len(lista_personas)
num_audios_sources = len([item for item in os.listdir(dir_sources_audios) if os.path.isfile(os.path.join(dir_sources_audios, item))])
#num_audios_por_persona = int(num_audios_sources / num_personas)

#creamos ficheros de log

dia = str(datetime.datetime.now())[:10].replace('-', '')
hora = str(datetime.datetime.now())[11:-7]
log_file_name = dir_log_audios + 'log_Reparto_' + timeID + '.xlsx'
log_file_name_summary = dir_log_audios + 'log_summary_' + timeID + '.log'

# creamos excel para el log

wb=Workbook()
wb.save(log_file_name)
wb=load_workbook(log_file_name)
sheet=wb.active

# Creamos las cabeceras de las columnas del log

sheet.cell(row=1, column=1).value = 'id'
sheet.cell(row=1, column=2).value = 'hora'
sheet.cell(row=1, column=3).value = 'persona'
sheet.cell(row=1, column=4).value = 'filename'
sheet.cell(row=1, column=5).value = 'new_file_name'
sheet.cell(row=1, column=6).value = 'dir_target'
sheet.cell(row=1, column=7).value = 'timestamp'
sheet.cell(row=1, column=8).value = 'dia'
sheet.cell(row=1, column=9).value = 'hora'

# distribuimos los ficheros a las carpetas de las personas correspondientes

j = 1 - num_audios_por_persona
for persona in lista_personas:
    
    #los ficheros se mueven a cada directorio de la persona en la lista
    dir_target = dir_sources_personas + persona + '/' + timeID
    i = 0
    j = j + num_audios_por_persona
    
    #ordenamos por 6 ultimos caracteres del fichero audio fuente
    for filename in sorted(os.listdir(dir_sources_audios), key=lambda x:x[-6:]):
        if i < num_audios_por_persona:
            if fnmatch.fnmatch(filename, '*.wav'):
                origen = dir_sources_audios + filename
                shutil.move(origen, dir_target)
                new_file_name = persona +'_' + filename
                os.rename(dir_target + '/' + filename, dir_target + '/' + new_file_name)
                i = i + 1
                #insertamos los valores de la asignación en el log
                sheet.cell(row=i+j, column=1).value = str(datetime.datetime.now()).replace('-', '').replace(' ', '_').replace(':', '').replace('.', '') 
                sheet.cell(row=i+j, column=2).value = hora
                sheet.cell(row=i+j, column=3).value = persona
                sheet.cell(row=i+j, column=4).value = filename
                sheet.cell(row=i+j, column=5).value = new_file_name
                sheet.cell(row=i+j, column=6).value = dir_target
                sheet.cell(row=i+j, column=7).value = str(datetime.datetime.now())
                sheet.cell(row=i+j, column=8).value = dia
                sheet.cell(row=i+j, column=9).value = hora

#guardamos el log
wb.save(log_file_name)


# Creamos las cabeceras de las columnas del log resumen

log_file_summary = open(log_file_name_summary, "w")
log_file_summary.write('Número de personas: ' + str(len(lista_personas)) + '; número de ficheros repartidos por persona: ' + str(num_audios_por_persona) + '; numero total de ficheros: ' + str(num_audios_sources) + '; número total de ficheros repartidos: ' + str(num_audios_por_persona * len(lista_personas)) + '; número de ficheros sin repartir: ' + str(num_audios_sources - (num_audios_por_persona * len(lista_personas))))
log_file_summary.close()

        